In [0]:
SELECT
  COUNT(*)                                   AS total,
  (COUNT(*) - COUNT(year))                   AS year_nulls,
  MIN(year)                                  AS min_year,
  MAX(year)                                  AS max_year
FROM movielens.silver.movies;

SELECT genre, COUNT(*) AS n_movies
FROM movielens.silver.movie_genres
GROUP BY genre
ORDER BY n_movies DESC;